# 解谜游戏：Akari(AKA: Light Up!)

- 在空白格子里放置灯，灯可以照亮横纵所有行，除非遇到黑色格子的阻碍；
- 黑色格子里的数字表示这个黑色格子上下左右四个位置的灯的数量

输入数据：棋盘（用W表示白色格子，B表示黑色格子，数字表示“既是黑色格子又是数字”的格子，棋盘的长、宽；

输出数据：放置的情况

- 约束条件：
    - 数字格的灯约束必须满足；
    - 对每个垂直 / 水平的连续白色格形成的宫，里面的灯泡不能超过1（不然就会存在互相照亮这种情况）
    - 对每个空白格子，搜索上下左右四个方向以及自己的位置（直到在所有方向都到达边界或者遇到黑色格子），这么多格子里必须有至少一盏灯（这个约束限制了每个白色格子都必须被照亮）

-------


- Place lights in the blank squares. Lights illuminate all rows and columns horizontally and vertically, unless obstructed by black squares.
The numbers in black squares indicate the number of lights in the four positions surrounding the black square: up, down, left, and right.
Input data: The chessboard (represented by 'W' for white squares, 'B' for black squares, and numbers indicating squares that are both black and numbered), as well as the length and width of the chessboard.

- Output data: The placement situation.

- Constraints:

    - The light constraints for numbered squares must be satisfied.
    - For each vertical/horizontal continuous group of white squares forming a block, there cannot be more than one light bulb inside (to avoid situations where lights illuminate each other).
    - For each blank square, search in the up, down, left, and right directions, as well as its own position (until reaching the boundary in all directions or encountering a black square). There must be at least one light among these squares (this constraint ensures that every white square must be illuminated).

In [1]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp


def akari_solver(grid, X, Y):
    
    try:
        len(grid) == X * Y
    except Exception:
        raise(f"检查输入棋盘的长度 length of grid != {X}*{Y}")
    
    def akari_cross_neighbours(pos_x, pos_y):
    # Cross site
        res = []
        if pos_x + 1 < Y and grid[(pos_x + 1) * Y + pos_y] == "W":
            res.append(x[pos_x + 1, pos_y])
        
        if pos_x - 1 >= 0 and grid[(pos_x - 1) * Y + pos_y] == "W":
            res.append(x[pos_x - 1, pos_y])
        
        if pos_y + 1 < X and grid[pos_x * Y + pos_y + 1] == "W":
            res.append(x[pos_x, pos_y + 1])
        
        if pos_y - 1 >= 0 and grid[pos_x * Y + pos_y - 1] == "W":
            res.append(x[pos_x, pos_y - 1])

        
        return res
    
    def akari_neighbours(pos_x , pos_y):
        result = []
        ticks = 1
        while pos_y + ticks < X and grid[pos_x  * X + pos_y + ticks] == "W":
            result.append(x[pos_x, pos_y + ticks])
            ticks += 1
        ticks = 1
        while pos_y - ticks >= 0 and grid[pos_x  * X + pos_y - ticks] == "W":
            result.append(x[pos_x, pos_y - ticks])
            ticks += 1
        
        ticks = 1
        while pos_x - ticks >= 0 and grid[(pos_x - ticks) * X + pos_y] == "W":
            result.append(x[pos_x - ticks, pos_y])
            ticks += 1
        
        ticks = 1
        while pos_x + ticks < Y and grid[(pos_x + ticks) * X + pos_y] == "W":
            result.append(x[pos_x + ticks, pos_y])
            ticks += 1
        
        result.append(x[pos_x, pos_y])
        return result


    model = cp.CpModel()
    x = {}
    grid_str = ["" for i in range(Y)]
    
    for i in range(Y):
        for j in range(X):
            if grid[i * Y + j] == "W":
                x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
            else:
                x[i, j] = 0
            grid_str[i] += grid[i * Y + j]
            
    for i in range(Y):
        tmp_cabin = []
        for j in range(X):
            if grid_str[i][j] == "W":
                tmp_cabin.append(x[i, j])
                if j == X - 1:
                    model.Add(sum(tmp_cabin) <= 1)
            else:
                if len(tmp_cabin) > 0:
                    model.Add(sum(tmp_cabin) <= 1)
                tmp_cabin = []
                if grid_str[i][j] in "0123456789":
                    light_constr = akari_cross_neighbours(i, j)
                    model.Add(sum(light_constr) == int(grid_str[i][j]))
                
    for j in range(X):
        tmp_cabin = []
        for i in range(Y):
            if grid_str[i][j] == "W":
                tmp_cabin.append(x[i, j])
                if i == Y - 1:
                    model.Add(sum(tmp_cabin) <= 1)
            elif len(tmp_cabin) > 0:
                model.Add(sum(tmp_cabin) <= 1)
                tmp_cabin = []
            

    # Constraints: Lights in every small cage must <= 1
        # print(f"{pos_x}, {pos_y}")
        # print(result)
        # return result
        
    for idx, c in enumerate(grid):
        
        if c == "W":
            tobe_lighted = akari_neighbours(idx // X, idx % X)
            model.Add(sum(tobe_lighted) >= 1)
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
        
    

if __name__ == "__main__":
    
    grid = "".join([
        "BWWW2WWWWWWWBWWWB",
        "WWHWWWWHWHWWWW1WW",
        "W1WWW2WWWWW2WWWBW",
        "WWWBWWWWBWWWW2WWW",
        "1WWW2WWWWWWWBWWWB",
        "WW2WWWWBWBWWWW2WW",
        "W1WWBWBWWWBWBWWBW",
        "WWW1WWWWWWWWWBWWW",
        "0WWWWBWWWWW2WWWW0",
        "WWW1WWWWWWWWWBWWW",
        "W1WWBWBWWW1WBWW1W",
        "WWBWWWW2W1WWWW1WW",
        "1WWW2WWWWWWWBWWWB",
        "WWW1WWWW0WWWW3WWW",
        "W1WWW2WWWWW1WWW1W",
        "WW1WWWW1WBWWWWBWW",
        "BWWW1WWWWWWW1WWWB"
    ])
    # pass 
    akari_solver(grid , 17, 17)

0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 
1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 
0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 
0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 
0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 
0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 
0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 
1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 
0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 
0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 
1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 
0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 

NumConflicts: 0
NumBranches: 0
WallTime: 0.008501



![](../assets/figures/Atari.png)


![](../assets/figures/Atari1.png)


> Special Thanks to This Bilibili UP [This video](https://www.bilibili.com/video/BV1nE411s7cx/)
>
> And [This Github repo](https://github.com/Borroot/akari)'s help.